In [1]:
import numpy as np
import pandas as pd

names = ["userid", "time", "bank_id", "pre_amount_of_bill", "pre_repayment", "credit_amount", \
         "amount_of_bill_left", "least_repayment", "consume_amount", "amount_of_bill", "adjust_amount", \
         "circ_interest", "avail_amount", "prepare_amount", "repayment_state"]

bill_train = pd.read_csv("../../pcredit/train/bill_detail_train.txt", header=None)
bill_test = pd.read_csv("../../pcredit/test/bill_detail_test.txt", header=None)

bill_data = pd.concat([bill_train, bill_test])
bill_data.columns = names

In [2]:
bill_data.head()

,userid,time,bank_id,pre_amount_of_bill,pre_repayment,credit_amount,amount_of_bill_left,least_repayment,consume_amount,amount_of_bill,adjust_amount,circ_interest,avail_amount,prepare_amount,repayment_state
0,3150,5906744363,6,18.626118,18.661937,20.664418,18.905766,17.847133,1,0.0,0.0,0.0,0.0,19.971271,0
1,3150,5906744401,6,18.905766,18.909954,20.664418,19.113305,17.911506,1,0.0,0.0,0.0,0.0,19.971271,0
2,3150,5906744427,6,19.113305,19.150290,20.664418,19.300194,17.977610,1,0.0,0.0,0.0,0.0,19.971271,0
3,3150,5906744515,6,19.300194,19.300280,21.000890,20.303240,18.477177,1,0.0,0.0,0.0,0.0,20.307743,0
4,3150,5906744562,6,20.303240,20.307744,21.000890,20.357134,18.510985,1,0.0,0.0,0.0,0.0,20.307743,0


In [3]:

stage = ['stg1_','stg2_','stg3_','stg4_','stg5_','stg6_','stg7_','stg8_','stg9_','stg10_']

cols = ['pre_amount_of_bill', 'pre_repayment', 'credit_amount',
        'amount_of_bill_left', 'least_repayment', 'consume_amount',
        'amount_of_bill', 'circ_interest', 'avail_amount',
        'prepare_amount']
sts = ['_min', '_max', '_median', '_mean', '_std', '_cnt', '_max_min']


features = pd.DataFrame(columns=['userid'] + [ st+s+p for p in sts  for st in stage for s in cols])

In [21]:
def test_(col):
    #  首先去除缺失值，获得分段
    data = bill_data[bill_data[col] != 0]['userid',col]
    print 'hello'
    #  获得 10% 20% 到 90% 的数据分割点
    t = data[col].describe(percentiles=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

    split_point = [0, int(t['10%']), int(t['20%']), int(t['30%']), int(t['40%']), int(t['50%']), int(t['60%']),
                   int(t['70%']), int(t['80%']), int(t['90%']), 1e11]

    # 获得所有的用户
    users = list(data.userid.unique())

    ft = [st+col+s  for s in sts for st in stage]
    ftures = pd.DataFrame(columns=ft)

    i = 0

    for u in users:

        d = {'userid':u}
        bill_user = bill_data[bill_data.userid==u]
        i += 1

        if i>2: break
        for i in range(10):
                stg = stage[i]
                di = bill_user[(split_point[i]<bill_user[col])&(bill_user[col]<split_point[i+1])]
                #  计算每一个用户的 最小，最大，中值，平均值，方差，数量，以及最大值和最小值的差
                d[stg+col+'_min'] = di[col].min()
                d[stg+col+'_max'] = di[col].max()
                d[stg+col+'_median'] = di[col].median()
                d[stg+col+'_mean'] = di[col].mean()
                d[stg+col+'_std'] = di[col].std()
                d[stg+col+'_cnt'] = di[col].count()
                d[stg+col+'_max_min'] = di[col].max() - di[col].min()
        this_tv_features = pd.DataFrame(d,index=[0])
        ftures = pd.concat([ftures,this_tv_features])
    print ftures.head()
    return ftures

from multiprocessing import Pool
pool = Pool(12)
rst = []
for col in ['pre_amount_of_bill']:
    print col
    rst.append(pool.apply_async(test_,args=(col,)))
pool.close()
pool.join()

pre_amount_of_bill


In [24]:
str(rst[0].get())

KeyError: ('userid', 'pre_amount_of_bill')

In [14]:
from multiprocessing import Pool
pool = Pool(12)
rst = []
for col in ['pre_amount_of_bill']:
    print col
    rst.append(pool.apply_async(test_,args=(col,)))
pool.close()
pool.join()

pre_amount_of_bill


In [19]:
[ i.get([ 'userid','pre_amount_of_bill']) for i in rst]

KeyError: ('userid', 'pre_amount_of_bill')